In [1]:
pip install transformers accelerate datasets

In [2]:
from datasets import load_dataset
multi_news = load_dataset("multi_news", split = 'test')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
multi_news.to_pandas()

,document,summary
0,GOP Eyes Gains As Voters In 11 States Pick Gov...,– It's a race for the governor's mansion in 11...
1,\n \n \n \n UPDATE: 4/19/2001 Read Richard Met...,– It turns out Facebook is only guilty of abou...
2,It's the Golden State's latest version of the ...,– Not a big fan of Southern California? Neithe...
3,The seed for this crawl was a list of every ho...,– Why did Microsoft buy Nokia's phone business...
4,After a year in which liberals scored impressi...,– The Supreme Court is facing a docket of high...
...,...,...
5617,Tweet with a location \n \n You can add locati...,– The traditional end-of-summit group photo at...
5618,Loic Venance/AFP/Getty Images \n \n The awards...,– Sofia Coppola scored a historic victory at t...
5619,(CNN) A federal criminal investigation into a ...,– The duck boat sinking that killed 17 on a Mi...
5620,An archive of the public statements deleted by...,– Note to tweeting politicians: Watch what you...


In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('t5-small')

In [5]:
multi_news = multi_news.train_test_split(test_size = .2)

In [6]:
prefix = "summarize: "

def processfunction(examples):
    # Add the prefix and generate inputs
    inputs = [prefix + doc for doc in examples["document"]]

    # Tokenize inputs
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    # Tokenize labels (summaries)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs


In [7]:
tokenized_multi_news = multi_news.map(processfunction, batched = True)

Map:   0%|          | 0/4497 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1125 [00:00<?, ? examples/s]

In [8]:
from transformers import DataCollatorForSeq2Seq, AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")

# Data collator for padding
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="t5-small")

# Load the model
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")


In [9]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [10]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model= model,
    args=training_args,
    train_dataset=tokenized_multi_news['train'],
    eval_dataset=tokenized_multi_news['test'],
    tokenizer=tokenizer,
    data_collator=data_collator
)


In [11]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,2.833870
2,3.262700,2.767928
3,3.009200,2.742691
4,2.964500,2.724673
5,2.921900,2.714046
6,2.910600,2.706007
7,2.896200,2.701548
8,2.889500,2.697668
9,2.874100,2.695912
10,2.874800,2.695184


TrainOutput(global_step=4500, training_loss=2.9559378526475695, metrics={'train_runtime': 2920.8546, 'train_samples_per_second': 15.396, 'train_steps_per_second': 1.541, 'total_flos': 1.217264162439168e+16, 'train_loss': 2.9559378526475695, 'epoch': 10.0})

In [13]:
Text = """The Evolution of Technology and Its Impact on Society

Since the dawn of civilization, human beings have continuously sought to develop tools and systems that simplify their lives, increase productivity, and unlock new potential. From the invention of the wheel and the plow in ancient Mesopotamia to the sophisticated artificial intelligence systems of the 21st century, the trajectory of technological evolution has been nothing short of transformative.

The first major technological revolution came with the Agricultural Revolution, which enabled early humans to shift from nomadic hunting and gathering to settled farming. This transition allowed for population growth, the rise of cities, and the establishment of governance systems. Technology during this era focused primarily on improving farming tools, irrigation methods, and food storage systems. Innovations like the sickle and the use of oxen for plowing were key drivers of agricultural efficiency.

The next significant leap came during the Industrial Revolution, beginning in the late 18th century. Fueled by the development of steam power, mechanized manufacturing, and advances in metallurgy, the Industrial Revolution marked a shift from manual labor to machine-based production. This era brought with it railroads, telegraphs, and factories, fundamentally changing how people lived and worked. Urbanization accelerated, and economies transitioned from agrarian to industrial."""


In [14]:
input_ids = tokenizer(Text,max_length = 1024, truncation = True, return_tensors = "pt").input_ids
input_ids = input_ids.to('cuda')

In [4]:
import torch

with torch.no_grad():
  if model.device.type =="cuda":
    output = model.generate(input_ids, max_length = 128, num_beams = 5)
summary_ids = output[0].tolist()

summary_ids = tokenizer.decode(summary_ids, skip_special_tokens = True)
print(summary_ids)

NameError: name 'model' is not defined

In [18]:
ref_summary = "The evolution of technology has profoundly shaped human society, starting from ancient tools like the wheel and plow to modern AI systems. The Agricultural Revolution enabled settled farming, spurring urban growth and governance structures through improved farming tools. Later, the Industrial Revolution introduced mechanized production, steam power, and railroads, transforming economies and accelerating urbanization."

In [3]:
from rouge import Rouge
rouge = Rouge()
scores = rouge.get_scores(summary_ids, ref_summary)
scores

NameError: name 'summary_ids' is not defined